In [1]:
import numpy as np
import dreamcoder as dc
from dreamcoder.domains.quantum_algorithms.primitives import *
from dreamcoder.domains.quantum_algorithms.tasks import *
from qiskit.transpiler.synthesis import solovay_kitaev
skd = solovay_kitaev.SolovayKitaevDecomposition()

import time
from tqdm import trange
import random
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

In [2]:
# This branch of qiskit is required:  pip install git+https://github.com/LNoorl/qiskit-terra.git@feature/sk-pass

In [3]:
basis_gates=['h',"cx",'t',"tdg"]

In [7]:
with QiskitTester(1) as QT:
    QT.circuit.t(0)
    QT.circuit.t(0)
qk.circuit.equivalence_library.StandardEquivalenceLibrary.add_equivalence(qk.circuit.library.SGate(),QT.circuit)

with QiskitTester(1) as QT:
    QT.circuit.tdg(0)
    QT.circuit.tdg(0)
qk.circuit.equivalence_library.StandardEquivalenceLibrary.add_equivalence(qk.circuit.library.SdgGate(),QT.circuit)

class ParametricSubstitution(qk.transpiler.TransformationPass):
    def run(self, dag):
        # iterate over all operations
        for node in dag.op_nodes():

            # if we hit a RYY or RZZ gate replace it
            if node.op.name in ["p"] and node.op.params[0]==np.pi/2:

                # calculate the replacement
                replacement = QuantumCircuit(1)
                replacement.s([0])

                # replace the node with our new decomposition
                dag.substitute_node_with_dag(node, circuit_to_dag(replacement))
        return dag
pm = qk.transpiler.PassManager()
pm.append(ParametricSubstitution())

### Some fixed-size circuits

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.x(QT.q(0))
QT.circuit.draw(),qk.transpile(QT.circuit, backend, basis_gates).draw()
QT.circuit=pm.run(QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
transpiled.draw(),discretized.draw()

(   ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐
 q: ┤ H ├┤ T ├┤ T ├┤ T ├┤ T ├┤ H ├
    └───┘└───┘└───┘└───┘└───┘└───┘,
    ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐
 q: ┤ H ├┤ T ├┤ T ├┤ T ├┤ T ├┤ H ├
    └───┘└───┘└───┘└───┘└───┘└───┘)

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.y(QT.q(0))
QT.circuit.draw(),qk.transpile(QT.circuit, backend, basis_gates).draw()
QT.circuit=pm.run(QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
transpiled.draw(),discretized.draw()

(global phase: π/2
    ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐
 q: ┤ T ├┤ T ├┤ T ├┤ T ├┤ H ├┤ T ├┤ T ├┤ T ├┤ T ├┤ H ├
    └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘,
 global phase: π/2
    ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐
 q: ┤ T ├┤ T ├┤ T ├┤ T ├┤ H ├┤ T ├┤ T ├┤ T ├┤ T ├┤ H ├
    └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘)

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.z(QT.q(0))
QT.circuit=pm.run(QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

(   ┌───┐
 q: ┤ Z ├
    └───┘,
    ┌───┐┌───┐┌───┐┌───┐
 q: ┤ T ├┤ T ├┤ T ├┤ T ├
    └───┘└───┘└───┘└───┘,
    ┌───┐┌───┐┌───┐┌───┐
 q: ┤ T ├┤ T ├┤ T ├┤ T ├
    └───┘└───┘└───┘└───┘)

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.t(QT.q(0))
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(QT.circuit)
QT.circuit.draw(),discretized.draw()
QT.result

array([[1.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.70710678+0.70710678j]])

In [69]:
with QiskitTester(1) as QT:
    QT.circuit.s(0)
QT.circuit=pm.run(QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

(   ┌───┐
 q: ┤ S ├
    └───┘,
    ┌───┐┌───┐
 q: ┤ T ├┤ T ├
    └───┘└───┘,
    ┌───┐┌───┐
 q: ┤ T ├┤ T ├
    └───┘└───┘)

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.append(qk.circuit.library.PhaseGate(np.pi/2),(0,))
QT.circuit = pm.run(QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

(   ┌───┐
 q: ┤ S ├
    └───┘,
    ┌───┐┌───┐
 q: ┤ T ├┤ T ├
    └───┘└───┘,
    ┌───┐┌───┐
 q: ┤ T ├┤ T ├
    └───┘└───┘)

In [71]:
with QiskitTester(1) as QT:
    QT.circuit.sx(QT.q(0))
QT.circuit=pm.run(QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
transpiled.draw(),discretized.draw()

(global phase: π/4
    ┌─────┐┌─────┐┌───┐┌─────┐┌─────┐
 q: ┤ Tdg ├┤ Tdg ├┤ H ├┤ Tdg ├┤ Tdg ├
    └─────┘└─────┘└───┘└─────┘└─────┘,
 global phase: π/4
    ┌─────┐┌─────┐┌───┐┌─────┐┌─────┐
 q: ┤ Tdg ├┤ Tdg ├┤ H ├┤ Tdg ├┤ Tdg ├
    └─────┘└─────┘└───┘└─────┘└─────┘)

In [ ]:
with QiskitTester(2) as QT:
    QT.circuit.sxdg(QT.q(1))
QT.circuit = pm.run( QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

(     ┌──────┐
 q_0: ┤ √Xdg ├
      └──────┘
 q_1: ────────
              ,
 global phase: 7π/4
      ┌───┐┌───┐┌───┐┌───┐┌───┐
 q_0: ┤ T ├┤ T ├┤ H ├┤ T ├┤ T ├
      └───┘└───┘└───┘└───┘└───┘
 q_1: ─────────────────────────
                               ,
 global phase: 7π/4
      ┌───┐┌───┐┌───┐┌───┐┌───┐
 q_0: ┤ T ├┤ T ├┤ H ├┤ T ├┤ T ├
      └───┘└───┘└───┘└───┘└───┘
 q_1: ─────────────────────────
                               )

In [ ]:
with QiskitTester(2) as QT:
    QT.circuit.cy(QT.q(0),QT.q(1))
QT.circuit.draw(),qk.transpile(QT.circuit, backend, basis_gates).draw()
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

(     ┌───┐
 q_0: ┤ Y ├
      └─┬─┘
 q_1: ──■──
           ,
      ┌─────┐┌─────┐┌───┐┌───┐┌───┐
 q_0: ┤ Tdg ├┤ Tdg ├┤ X ├┤ T ├┤ T ├
      └─────┘└─────┘└─┬─┘└───┘└───┘
 q_1: ────────────────■────────────
                                   ,
      ┌─────┐┌─────┐┌───┐┌───┐┌───┐
 q_0: ┤ Tdg ├┤ Tdg ├┤ X ├┤ T ├┤ T ├
      └─────┘└─────┘└─┬─┘└───┘└───┘
 q_1: ────────────────■────────────
                                   )

In [76]:
with QiskitTester(2) as QT:
    QT.circuit.cz(QT.q(0),QT.q(1))
QT.circuit.draw(),qk.transpile(QT.circuit, backend, basis_gates).draw()
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
transpiled.draw(),discretized.draw()

(     ┌───┐┌───┐┌───┐
 q_0: ┤ H ├┤ X ├┤ H ├
      └───┘└─┬─┘└───┘
 q_1: ───────■───────
                     ,
      ┌───┐┌───┐┌───┐
 q_0: ┤ H ├┤ X ├┤ H ├
      └───┘└─┬─┘└───┘
 q_1: ───────■───────
                     )

In [78]:
with QiskitTester(2) as QT:
    QT.circuit.swap(QT.q(0),QT.q(1))
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

(        
 q_0: ─X─
       │ 
 q_1: ─X─
         ,
      ┌───┐     ┌───┐
 q_0: ┤ X ├──■──┤ X ├
      └─┬─┘┌─┴─┐└─┬─┘
 q_1: ──■──┤ X ├──■──
           └───┘     ,
      ┌───┐     ┌───┐
 q_0: ┤ X ├──■──┤ X ├
      └─┬─┘┌─┴─┐└─┬─┘
 q_1: ──■──┤ X ├──■──
           └───┘     )

In [ ]:
with QiskitTester(2) as QT:
    QT.circuit.iswap(QT.q(0),QT.q(1))
QT.circuit = pm.run( QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

(     ┌────────┐
 q_0: ┤1       ├
      │  Iswap │
 q_1: ┤0       ├
      └────────┘,
      ┌───┐┌───┐     ┌───┐     ┌───┐
 q_0: ┤ T ├┤ T ├─────┤ X ├──■──┤ H ├
      ├───┤├───┤┌───┐└─┬─┘┌─┴─┐└───┘
 q_1: ┤ T ├┤ T ├┤ H ├──■──┤ X ├─────
      └───┘└───┘└───┘     └───┘     ,
      ┌───┐┌───┐     ┌───┐     ┌───┐
 q_0: ┤ T ├┤ T ├─────┤ X ├──■──┤ H ├
      ├───┤├───┤┌───┐└─┬─┘┌─┴─┐└───┘
 q_1: ┤ T ├┤ T ├┤ H ├──■──┤ X ├─────
      └───┘└───┘└───┘     └───┘     )

In [79]:
with QiskitTester(3) as QT:
    QT.circuit.ccx(QT.q(0),QT.q(1),QT.q(2))
QT.circuit.draw(),qk.transpile(QT.circuit, backend, basis_gates).draw()
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

(     ┌───┐
 q_0: ┤ X ├
      └─┬─┘
 q_1: ──■──
        │  
 q_2: ──■──
           ,
      ┌───┐┌───┐┌─────┐┌───┐┌───┐┌───┐┌─────┐┌───┐┌───┐ ┌───┐      
 q_0: ┤ H ├┤ X ├┤ Tdg ├┤ X ├┤ T ├┤ X ├┤ Tdg ├┤ X ├┤ T ├─┤ H ├──────
      └───┘└─┬─┘└─────┘└─┬─┘└───┘└─┬─┘└┬───┬┘└─┬─┘├───┤┌┴───┴┐┌───┐
 q_1: ───────■───────────┼─────────■───┤ T ├───┼──┤ X ├┤ Tdg ├┤ X ├
                         │             └───┘   │  └─┬─┘└┬───┬┘└─┬─┘
 q_2: ───────────────────■─────────────────────■────■───┤ T ├───■──
                                                        └───┘      ,
      ┌───┐┌───┐┌─────┐┌───┐┌───┐┌───┐┌─────┐┌───┐┌───┐ ┌───┐      
 q_0: ┤ H ├┤ X ├┤ Tdg ├┤ X ├┤ T ├┤ X ├┤ Tdg ├┤ X ├┤ T ├─┤ H ├──────
      └───┘└─┬─┘└─────┘└─┬─┘└───┘└─┬─┘└┬───┬┘└─┬─┘├───┤┌┴───┴┐┌───┐
 q_1: ───────■───────────┼─────────■───┤ T ├───┼──┤ X ├┤ Tdg ├┤ X ├
                         │             └───┘   │  └─┬─┘└┬───┬┘└─┬─┘
 q_2: ───────────────────■─────────────────────■────■───┤ T ├───■──
                              

In [164]:
with QiskitTester(2) as QT:
    QT.circuit.dcx(QT.q(0),QT.q(1))
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

(     ┌──────┐
 q_0: ┤1     ├
      │  Dcx │
 q_1: ┤0     ├
      └──────┘,
      ┌───┐     
 q_0: ┤ X ├──■──
      └─┬─┘┌─┴─┐
 q_1: ──■──┤ X ├
           └───┘,
      ┌───┐     
 q_0: ┤ X ├──■──
      └─┬─┘┌─┴─┐
 q_1: ──■──┤ X ├
           └───┘)

In [158]:
with QiskitTester(2) as QT:
    QT.circuit.ch(QT.q(0),QT.q(1))
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

(     ┌───┐
 q_0: ┤ H ├
      └─┬─┘
 q_1: ──■──
           ,
      ┌───┐┌───┐┌───┐┌───┐┌─────┐┌───┐┌─────┐
 q_0: ┤ S ├┤ H ├┤ T ├┤ X ├┤ Tdg ├┤ H ├┤ Sdg ├
      └───┘└───┘└───┘└─┬─┘└─────┘└───┘└─────┘
 q_1: ─────────────────■─────────────────────
                                             ,
      ┌───┐┌───┐┌───┐┌───┐┌───┐┌─────┐┌───┐┌─────┐┌─────┐
 q_0: ┤ T ├┤ T ├┤ H ├┤ T ├┤ X ├┤ Tdg ├┤ H ├┤ Tdg ├┤ Tdg ├
      └───┘└───┘└───┘└───┘└─┬─┘└─────┘└───┘└─────┘└─────┘
 q_1: ──────────────────────■────────────────────────────
                                                         )

In [165]:
with QiskitTester(3) as QT:
    QT.circuit.cswap(QT.q(0),QT.q(1),QT.q(2))
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

(        
 q_0: ─X─
       │ 
 q_1: ─X─
       │ 
 q_2: ─■─
         ,
           ┌───┐┌───┐┌─────┐┌───┐┌───┐┌───┐┌─────┐┌───┐┌───┐ ┌───┐           
 q_0: ──■──┤ H ├┤ X ├┤ Tdg ├┤ X ├┤ T ├┤ X ├┤ Tdg ├┤ X ├┤ T ├─┤ H ├────────■──
      ┌─┴─┐└───┘└─┬─┘└─────┘└─┬─┘└───┘└─┬─┘└┬───┬┘└─┬─┘├───┤┌┴───┴┐┌───┐┌─┴─┐
 q_1: ┤ X ├───────■───────────┼─────────■───┤ T ├───┼──┤ X ├┤ Tdg ├┤ X ├┤ X ├
      └───┘                   │             └───┘   │  └─┬─┘└┬───┬┘└─┬─┘└───┘
 q_2: ────────────────────────■─────────────────────■────■───┤ T ├───■───────
                                                             └───┘           ,
           ┌───┐┌───┐┌─────┐┌───┐┌───┐┌───┐┌─────┐┌───┐┌───┐ ┌───┐           
 q_0: ──■──┤ H ├┤ X ├┤ Tdg ├┤ X ├┤ T ├┤ X ├┤ Tdg ├┤ X ├┤ T ├─┤ H ├────────■──
      ┌─┴─┐└───┘└─┬─┘└─────┘└─┬─┘└───┘└─┬─┘└┬───┬┘└─┬─┘├───┤┌┴───┴┐┌───┐┌─┴─┐
 q_1: ┤ X ├───────■───────────┼─────────■───┤ T ├───┼──┤ X ├┤ Tdg ├┤ X ├┤ X ├
      └───┘                   │             └───┘   │  └─┬─┘└┬───┬┘└─┬

In [64]:
# Bell
with QiskitTester(2) as QT:
    QT.circuit.h(QT.q(0))
    QT.circuit.cx(QT.q(0),QT.q(1))
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

(          ┌───┐
 q_0: ─────┤ X ├
      ┌───┐└─┬─┘
 q_1: ┤ H ├──■──
      └───┘     ,
           ┌───┐
 q_0: ─────┤ X ├
      ┌───┐└─┬─┘
 q_1: ┤ H ├──■──
      └───┘     ,
           ┌───┐
 q_0: ─────┤ X ├
      ┌───┐└─┬─┘
 q_1: ┤ H ├──■──
      └───┘     )

In [66]:
# GHZ
with QiskitTester(3) as QT:
    QT.circuit.h(QT.q(0))
    QT.circuit.cx(QT.q(0),QT.q(1))
    QT.circuit.cx(QT.q(0),QT.q(2))
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

(               ┌───┐
 q_0: ──────────┤ X ├
           ┌───┐└─┬─┘
 q_1: ─────┤ X ├──┼──
      ┌───┐└─┬─┘  │  
 q_2: ┤ H ├──■────■──
      └───┘          ,
                ┌───┐
 q_0: ──────────┤ X ├
           ┌───┐└─┬─┘
 q_1: ─────┤ X ├──┼──
      ┌───┐└─┬─┘  │  
 q_2: ┤ H ├──■────■──
      └───┘          ,
                ┌───┐
 q_0: ──────────┤ X ├
           ┌───┐└─┬─┘
 q_1: ─────┤ X ├──┼──
      ┌───┐└─┬─┘  │  
 q_2: ┤ H ├──■────■──
      └───┘          )

### Invalid

In [166]:
with QiskitTester(2) as QT:
    QT.circuit.csx(QT.q(0),QT.q(1))
QT.result
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

TranspilerError: "Unable to map source basis {('csx', 2)} to target basis {'cx', 'snapshot', 't', 'tdg', 'reset', 'delay', 'barrier', 'h', 'measure'} over library <qiskit.circuit.equivalence.EquivalenceLibrary object at 0x1506702481f0>."

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.p(np.pi/8,QT.q(0))
QT.circuit.draw()
QT.result

array([[1.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.92387953+0.38268343j]])

In [ ]:
# we should be able to transpile this one
with QiskitTester(1) as QT:
    QT.circuit.p(np.pi/8,0)
    QT.circuit.p(np.pi/8,0)
QT.circuit.draw()
QT.circuit=pm.run(QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

TranspilerError: "Unable to map source basis {('p', 1)} to target basis {'cx', 'snapshot', 't', 'tdg', 'reset', 'delay', 'barrier', 'h', 'measure'} over library <qiskit.circuit.equivalence.EquivalenceLibrary object at 0x1506702481f0>."

In [ ]:
with QiskitTester(4) as QT:
    QT.circuit.append(qk.circuit.library.C3XGate(), (0,1,2,3))
QT.circuit.draw()
# transpiled=qk.transpile(QT.circuit, backend, basis_gates)
# discretized = skd(transpiled)
# QT.circuit.draw(),transpiled.draw(),discretized.draw()

q_0: ──■──
       │  
q_1: ──■──
       │  
q_2: ──■──
     ┌─┴─┐
q_3: ┤ X ├
     └───┘

In [ ]:
with QiskitTester(2) as QT:
    QT.circuit.append(qk.circuit.library.SGate().control(1), (0,1))
dag = qk.converters.circuit_to_dag(QT.circuit)

In [ ]:
# This is CS, but can we transpile it?
with QiskitTester(2) as QT:
    QT.circuit.append(qk.circuit.library.PhaseGate(np.pi/2).control(1),(0,1))

In [ ]:
with QiskitTester(2) as QT:
    QT.circuit.csx(QT.q(0),QT.q(1))
QT.circuit = pm.run( QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

TranspilerError: "Unable to map source basis {('csx', 2)} to target basis {'cx', 'snapshot', 't', 'tdg', 'reset', 'delay', 'barrier', 'h', 'measure'} over library <qiskit.circuit.equivalence.EquivalenceLibrary object at 0x1506702481f0>."

In [172]:
with QiskitTester(4) as QT:
    QT.circuit.rcccx(QT.q(0),QT.q(1),QT.q(2),QT.q(3))
QT.result
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

TranspilerError: "Unable to map source basis {('u2', 1), ('u1', 1), ('cx', 2)} to target basis {'cx', 'snapshot', 't', 'tdg', 'reset', 'delay', 'barrier', 'h', 'measure'} over library <qiskit.circuit.equivalence.EquivalenceLibrary object at 0x1506702481f0>."

In [ ]:
with QiskitTester(2) as QT:
    QT.circuit.ecr(QT.q(0),QT.q(1))
QT.circuit = pm.run( QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

TranspilerError: "Unable to map source basis {('ecr', 2)} to target basis {'cx', 'snapshot', 't', 'tdg', 'reset', 'delay', 'barrier', 'h', 'measure'} over library <qiskit.circuit.equivalence.EquivalenceLibrary object at 0x1506702481f0>."